### **-- Excel File Generator for EDB Master School List --**

## **Click the "Play Button" on the left hand side to generate & download the master excel file.**

In [1]:
#!/usr/bin/env python3

import requests
import pandas as pd, numpy as np
from pandas import ExcelWriter
from bs4 import BeautifulSoup

from google.colab import files

edbSchListURL = 'https://www.edb.gov.hk/en/student-parents/sch-info/sch-search/schlist-by-district/school-list-{}.html'
district18 = {'cw' : 'Central & Western',
                'hke' : 'Hong Kong East',
                'i' : 'Islands',
                'sou' : 'Southern',
                'wch' : 'Wan Chai',
                'kc' : 'Kowloon City',
                'kt' : 'Kwun Tong',
                'sk' : 'Sai Kung',
                'ssp' : 'Sham Shui Po',
                'wts' : 'Wong Tai Sin',
                'ytm' : 'Yau Tsim & Mong Kok',
                'n' : 'North',
                'st' : 'Sha Tin',
                'tp' : 'Tai Po',
                'kwt' : 'Kwai Chung & Tsing Yi',
                'tw' : 'Tsuen Wan',
                'tm' : 'Tuen Mun',
                'yl' : 'Yuen Long'}

dfCols = ['Sch ID', 'Sch Name (EN)', 'Sch Name (ZH)', 'Address (EN)', 'Website', 'Tel.', 'Principal (EN)', 'Principal (ZH)', 'Sch Type', 'District']
df = pd.DataFrame(columns=dfCols, dtype=str)

for districtAlias in district18:
    tarURL = edbSchListURL.format(districtAlias)
    htmlContent = requests.get(tarURL)
    htmlContent.encoding = 'utf-8'

    soup = BeautifulSoup(htmlContent.text, 'html.parser')
    tblContent = soup.find_all('table', {'class': 'tablestyleA'})

    for schTypeTblContent in tblContent:
        count = 0
        district = ''
        schType = ''
        for trContent in schTypeTblContent.findChildren('tr'):
            count += 1
            if count == 2:
                tdContent = trContent.findChildren('td')
                district = tdContent[0].contents[1].strip()
                schType = tdContent[1].contents[1].strip()
            elif count == 3:
                if 'PRIVATE SECONDARY SCHOOLS' in schType or 'PRIMARY' in schType or 'SPECIAL' in schType or 'KINDERGARTEN' in schType or 'OTHERS' in schType:
                    continue

                dataList = trContent.contents[0].contents[1].text.split('\n')
                dataList = [x.replace('\t', '').replace('\r', '').replace('\r', '').replace('\xa0', ' ').strip() for x in dataList]

                schInfoList = [[]]
                for data in dataList[17:]:
                    schInfoList[len(schInfoList)-1].append(data)

                    if 'School No./Location ID' in data:
                        schInfoList.append(list(schInfoList[len(schInfoList)-1][-13:]))
                        schInfoList[len(schInfoList)-2] = schInfoList[len(schInfoList)-2][0:-13]

                schInfoList = schInfoList[1:]

                for schInfo in schInfoList:
                    url = ''
                    if len(schInfo) > 57:
                        url = schInfo[57].strip()

                    schName = schInfo[3].strip().replace(' SEC ', ' SECONDARY ').replace(' PRI ', ' PRIMARY ')
                    schName = schName.replace(' SCH', ' SCHOOL').replace(' SCHOOLOOL', ' SCHOOL')
                    schName = schName.replace(' COLL', ' COLLEGE').replace(' COLLEGEEGE', ' COLLEGE')
                    schName = schName.replace('(SEC SECT', '(SEC SECTION').replace(' SECTIONION', ' SECTION')
                    schName = schName.replace('(SEC ', '(SECONDARY ')
                    schName = schName.replace(' SECONDARY SC', ' SECONDARY SCHOOL').replace(' SCHOOLHOOL', ' SCHOOL')
                    schName = schName.replace('CO-EDU ENG', 'CO-EDUCATIONAL ENGLISH')
                    
                    principalName = schInfo[31].strip()
                    schID = schInfo[12].split(':')[1].split('/')[0].strip()
                    schTel = schInfo[17].split(':')[1].strip()[0:8]
                    if principalName != 'PENDING':
                        if True:
                            df = df.append({
                                'Sch ID' : schID,
                                'Sch Name (EN)' : schName,
                                'Sch Name (ZH)' : schInfo[8].strip(),
                                'Address (EN)' : schInfo[6].strip(),
                                'Website' : url,
                                'Tel.' : schTel,
                                'Principal (EN)' : schInfo[31].strip(),
                                'Principal (ZH)' : schInfo[33].strip(),
                                'Sch Type' : schType,
                                'District' : district
                            }, ignore_index=True)

xlsFileName = 'EDB Master Secondary School List.xlsx'
df.to_excel(xlsFileName, 'School List', index=True)
files.download(xlsFileName)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>